In [33]:
from openai import OpenAI
import openai

import instructor 
from pydantic import BaseModel, Field

In [13]:
prompt = '''
you are a helpful assistant that can answer questions 
and help with tasks.'''

In [34]:
# client_OAI = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": prompt}]
)
response

ChatCompletion(id='chatcmpl-CWRL3dPpx9TuuvyJhEFrtLyy8rZRR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1761847621, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=7, prompt_tokens=23, total_tokens=30, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

## Instructor

In [35]:
client = instructor.from_openai(openai.OpenAI())

In [20]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="Answer to the user's question")


In [36]:
response, raw_response = client.chat.completions.create_with_completion(\
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": prompt}],
    response_model = RAGGenerationResponse
)
response.model_dump()



{'answer': 'The Grand Canyon is estimated to be around 5 to 6 million years old, formed by the erosion of the Colorado River cutting through layers of rock over millions of years.'}

In [27]:
raw_response

ChatCompletion(id='chatcmpl-CWRHdhH46413LlQxLoTvMVqX6G6Pu', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_nePVRtFpjZLSoX05AbawlLzB', function=Function(arguments='{"answer":"Can you tell me about the significance of the animal spirit in Native American cultures?"}', name='RAGGenerationResponse'), type='function')]))], created=1761847409, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_51db84afab', usage=CompletionUsage(completion_tokens=20, prompt_tokens=90, total_tokens=110, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

## RAG with instructor

In [42]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(url="http://localhost:6333")
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_images = []
    retrieved_metadata = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)
        
        # Extract image URL if available
        image_url = result.payload.get("image", None)
        retrieved_images.append(image_url)
        
        # Extract other metadata
        metadata = {
            "parent_asin": result.payload.get("parent_asin"),
            "price": result.payload.get("price"),
            "average_rating": result.payload.get("average_rating"),
            "rating_number": result.payload.get("rating_number"),
            "store": result.payload.get("store"),
            "video": result.payload.get("video"),
            "image": result.payload.get("image")
        }
        retrieved_metadata.append(metadata)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
        "retrieved_images": retrieved_images,
        "retrieved_metadata": retrieved_metadata,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):
        formatted_context += f"- {id}: {chunk}\n"

    return formatted_context



def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt



def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
        response_model=RAGGenerationResponse
    )

    return response



def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    # Filter out None values from images and create a list of valid image URLs
    valid_images = [img for img in retrieved_context["retrieved_images"] if img is not None]
    
    # Filter metadata to only include items with images
    valid_metadata = [meta for meta, img in zip(retrieved_context["retrieved_metadata"], retrieved_context["retrieved_images"]) if img is not None]

    final_result = {
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"],
        "images": valid_images,
        "product_metadata": valid_metadata
    }

    return final_result

In [43]:
response = rag_pipeline("What is the price of the product?", qdrant_client)

In [44]:
response

{'answer': 'The available products do not provide information about the price of the products.',
 'question': 'What is the price of the product?',
 'retrieved_context_ids': ['B09QM776N9',
  'B0BLRGWXQL',
  'B09VN35T1M',
  'B09RLSF7B2',
  'B0C649RHHN'],
 'retrieved_context': ["Bruno Marc Men's Fashion Sneakers Casual Comfort Canvas Skate Shoes 100% Canvas Imported Rubber sole Premium canvas upper is comfortable and soft to the touch and also provide advanced upper stitching workmanship make the men shoes durable enough to last a long time Soft inner material can provide a comfortable foot environment and prevent feet from grinding MEMORY FOAM INSOLE: A memory foam insole provides cushioning and reduces fatigue. The honeycomb design on the bottom of the insole keeps your feet dry and comfortable all day long Rubber-plastic foam outsole is not only soft and lightweight, but also has the characteristics of rubber anti-skid and wear-resistant. Three-dimensional non-slip textured sole can en